## 模型更新
如果一开始训练的模型数据较少,随着时间推移,有了新的数据,又不想重新训练新模型的时候,就用增量模型训练

In [1]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.models import model_from_json
from sklearn.model_selection import train_test_split

# 设定随机种子
seed = 7
np.random.seed(seed)

# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

x_train, x_increment, Y_train, Y_increment = train_test_split(x, Y, test_size=0.2, random_state=seed)

# Convert labels to categorical one-hot encoding
Y_train_labels = to_categorical(Y_train, num_classes=3)


# 构建模型函数
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# 构建模型
model = create_model()
model.fit(x_train, Y_train_labels, epochs=10, batch_size=5, verbose=2)

scores = model.evaluate(x_train, Y_train_labels, verbose=0)
print('Base %s: %.2f%%' % (model.metrics_names[1], scores[1] * 100))

# 模型保存成Json文件
model_json = model.to_json()
with open('src/model.increment.json', 'w') as file:
    file.write(model_json)

# 保存模型的权重值
model.save_weights('src/model.increment.json.h5')


# 从Json加载模型
with open('src/model.increment.json', 'r') as file:
    model_json = file.read()

# 加载模型
new_model = model_from_json(model_json)
new_model.load_weights('src/model.increment.json.h5')

# 编译模型
new_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# 增量训练模型
# Convert labels to categorical one-hot encoding
Y_increment_labels = to_categorical(Y_increment, num_classes=3)
new_model.fit(x_increment, Y_increment_labels, epochs=10, batch_size=5, verbose=2)
scores = new_model.evaluate(x_increment, Y_increment_labels, verbose=0)
print('Increment %s: %.2f%%' % (model.metrics_names[1], scores[1] * 100))

Using TensorFlow backend.
W0407 18:42:46.566566 140451463640896 deprecation_wrapper.py:119] From /home/liyuan3970/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/10
 - 0s - loss: 1.0141 - accuracy: 0.4000
Epoch 2/10
 - 0s - loss: 0.9718 - accuracy: 0.4750
Epoch 3/10
 - 0s - loss: 0.9392 - accuracy: 0.5500
Epoch 4/10
 - 0s - loss: 0.9055 - accuracy: 0.5833
Epoch 5/10
 - 0s - loss: 0.8717 - accuracy: 0.5833
Epoch 6/10
 - 0s - loss: 0.8306 - accuracy: 0.6000
Epoch 7/10
 - 0s - loss: 0.7886 - accuracy: 0.5500
Epoch 8/10
 - 0s - loss: 0.7531 - accuracy: 0.5000
Epoch 9/10
 - 0s - loss: 0.7268 - accuracy: 0.5250
Epoch 10/10
 - 0s - loss: 0.7055 - accuracy: 0.7750
Base accuracy: 80.00%
Epoch 1/10
 - 0s - loss: 0.8117 - accuracy: 0.7333
Epoch 2/10
 - 0s - loss: 0.8026 - accuracy: 0.7333
Epoch 3/10
 - 0s - loss: 0.7964 - accuracy: 0.7333
Epoch 4/10
 - 0s - loss: 0.7913 - accuracy: 0.7333
Epoch 5/10
 - 0s - loss: 0.7886 - accuracy: 0.7667
Epoch 6/10
 - 0s - loss: 0.7835 - accuracy: 0.7667
Epoch 7/10
 - 0s - loss: 0.7786 - accuracy: 0.8333
Epoch 8/10
 - 0s - loss: 0.7746 - accuracy: 0.8667
Epoch 9/10
 - 0s - loss: 0.7703 - accuracy: 0.8667
Epoch 10

## 神经网络的检查点保存机制
一般不会用到,如果模型过于大,可以一个epoch保存一个模型,然后训练出错的时候可以从某一个模型导入

In [8]:
from sklearn import datasets
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

# 导入数据
dataset = datasets.load_iris()

x = dataset.data
Y = dataset.target

# Convert labels to categorical one-hot encoding
Y_labels = to_categorical(Y, num_classes=3)

# 设定随机种子
seed = 7
np.random.seed(seed)
# 构建模型函数
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # 构建模型
    model = Sequential()
    model.add(Dense(units=4, activation='relu', input_dim=4, kernel_initializer=init))
    model.add(Dense(units=6, activation='relu', kernel_initializer=init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=init))

    # 编译模型
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# 构建模型
model = create_model()

# 设置检查点
filepath = 'src/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.h5'
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callback_list = [checkpoint]
model.fit(x, Y_labels, validation_split=0.2, epochs=20, batch_size=5, verbose=0, callbacks=callback_list)

/home/liyuan3970/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
